In [4]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import psycopg2

In [2]:
import numpy as np
import re

In [3]:

conn = psycopg2.connect("dbname=top100 user=postgres password=im4uema3cohb")

In [28]:
cur = conn.cursor()
command = "DROP TABLE IF EXISTS music;"
cur.execute(command)
command = "DROP TABLE IF EXISTS artist;"
cur.execute(command)
cur.close()

In [29]:
#table paroles avec foreign key nom musique et nom artiste
commands = (
        """
CREATE TABLE artist(
   id_artist INT GENERATED ALWAYS AS IDENTITY,
   nameArtist VARCHAR(100),
   birth DATE,
   city VARCHAR(100),
   country VARCHAR(100),
   PRIMARY KEY(id_artist)
)
        """,
        """
CREATE TABLE music(
   id_music INT GENERATED ALWAYS AS IDENTITY,
   id_artist INT,
   title VARCHAR(100),
   lyrics VARCHAR(1000),
   genre VARCHAR(100),
   releaseDate DATE,
   ranking INT,
   PRIMARY KEY(id_music),
   CONSTRAINT fk_artist
      FOREIGN KEY(id_artist) 
	  REFERENCES artist(id_artist)
	  ON DELETE CASCADE
)
        """)


In [30]:
cur = conn.cursor()
# create table one by one
for command in commands:
        cur.execute(command)
# close communication with the PostgreSQL database server
cur.close()
# commit the changes
conn.commit()

In [ ]:
sql2 = '''COPY details(employee_id,employee_name,\
employee_email,employee_salary)
FROM '/private/tmp/details.csv'
DELIMITER ','
CSV HEADER;'''
  
cursor.execute(sql2)

In [25]:
if conn is not None:
    conn.close()

In [4]:
import os
import pandas as pd

In [5]:
files = os.listdir("../chart")

dfs_chart = {}
for f in files:
    year = int(f.split("_")[0])
    month = f.split("_")[1].split(".")[0]

    if dfs_chart.get(year) is None:
        dfs_chart[year] = {}
        
    dfs_chart[year][month] = pd.read_csv("../chart/" + f)

In [6]:
dfs_chart[2004]["Juillet"]

,Rank,Artist,Music,Naissance,Pays d'origine,Origine,Nationalité,Pays
0,1,O-zone,Dragostea Din Tei,,Moldavie,,,Moldavie
1,2,K'maro,Femme Like U,,Canada,,,Canada
2,3,Calogero,Face À La Mer,"30 juillet 1971 (51 ans)Échirolles, France",,,Française,
3,4,T-rio,Mamae Eu Quero (choopeta),,,,,
4,5,Le 6-9,Le Poulailler,,,,,"France, Suisse"
...,...,...,...,...,...,...,...,...
95,96,Sandra Lou,Le Banana Split,,,,Française,
96,97,Mickey 3d,Johnny Rep,,France,,,France
97,98,Jean-luc Lahaye,On N'se Reverra Plus,"23 décembre 1952 (69 ans)Paris, France",,,,
98,99,Lmc,Take Me To The Clouds Above,,,,,


In [100]:
def test(row):
    match = re.match('.*([1-3][0-9]{3})', row)
    if match is not None:
        match = re.sub('ou.*', "", match.group())
    


In [78]:

dfs_chart[2020]["Fevrier"][dfs_chart[2020]["Fevrier"]['Naissance'].str.contains("10 mai 1995")]

,Rank,Artist,Music,Naissance,Pays d'origine,Origine,Nationalité,Pays
11,12,Aya Nakamura,40%,10 mai 1995 ou 10 mai 1995Bamako,,,Malienne,
69,70,Aya Nakamura,Pookie,10 mai 1995 ou 10 mai 1995Bamako,,,Malienne,


In [101]:
dfs_chart[2020]["Fevrier"]['Naissance'].apply(lambda row : test(row))

0     None
1     None
2     None
3     None
4     None
      ... 
95    None
96    None
97    None
98    None
99    None
Name: Naissance, Length: 100, dtype: object

In [131]:
months = {
    "janvier": "01",
    "février": "02", 
    "mars": "03", 
    "avril": "04", 
    "mai": "05", 
    "juin": "06", 
    "juillet": "07", 
    "août": "08", 
    "septembre": "09", 
    "octobre": "10", 
    "novembre": "11", 
    "décembre": "12",
    "1er": "1"}

In [8]:
#chercher mois et faire traitement
def insertNaissance2(row):
    texte = re.sub("[\(\[].*?[\)\]]", "", row['Naissance'])
    texte = texte.split()
    #print(texte)
    res = "0000-01-01"
    if len(texte)>2 and texte[0].isnumeric() and texte[2].isnumeric():
        res = str(texte[2])+"-"+str(months[texte[1]])+"-"+str(texte[0])
    return res

In [81]:
def pays(row):
    if row["Pays"] != " ":
        return row["Pays"]
    elif row["Nationalité"] != " ":
        return row["Nationalité"]
    elif row["Origine"] != " ":
        return row["Origine"]
    elif row["Pays d'origine"] != " ":
        return row["Pays d'origine"]
    else:
        texte = re.sub("[\(\[].*?[\)\]]", "", row['Naissance'])
        texte = texte.split()
        if len(texte)>3:
            return texte[-1]
        else:
            return ""

## Recherche Naissance

In [132]:
def insertNaissance(row):
    match = re.match('.*([1-2][0-9]{3})', row['Naissance'])
    if match is not None:
        match = re.sub('ou.*', "", match.group())
        match = " ".join(months.get(ele, ele) for ele in match.split())
 
    return match

In [133]:

for years in dfs_chart:
     for table in dfs_chart[years].values():
          to_insert = table.drop(['Rank', 'Music'], axis=1)
          to_insert['Naissance_to_insert'] = to_insert.apply(lambda row : insertNaissance(row), axis=1)
          #to_insert['Pays'] = to_insert['Pays'].astype("string")
          #to_insert['Pays_to_insert'] = to_insert.apply(lambda row : pays(row), axis=1)
          #print(to_insert['Pays_to_insert'].unique())

In [134]:
to_insert

,Artist,Naissance,Pays d'origine,Origine,Nationalité,Pays,Naissance_to_insert
0,Gazo,5 août 1994 (28 ans)Châteauroux,,,Française,,5 08 1994
1,Lomepal,"22 novembre 1988 (34 ans)Barcelone, Catalogne(...",,,,,22 11 1988
2,Lomepal,"22 novembre 1988 (34 ans)Barcelone, Catalogne(...",,,,,22 11 1988
3,Soolking,10 décembre 1989 (32 ans)Staouali (d),,,Algérienne,,10 12 1989
4,Rema,"1er mai 2000 (22 ans)Benin City, État d’Edo, ...",,,,,1 05 2000
...,...,...,...,...,...,...,...
95,Sdm,"28 novembre 1995 (26 ans)Meudon, France",,,,,28 11 1995
96,Lizzo,"27 avril 1988 (34 ans)Détroit, Michigan, État...",,,,,27 04 1988
97,Djadja et Dinaz,,France,,,France,None
98,Ckay,27 novembre 1997[réf. nécessaire],,,,,27 11 1997


In [13]:
to_insert['Pays_to_insert'].unique()

array(['Française', 'Catalogne', 'Algérienne', 'Nigeria', 'Belgique',
       'France', 'Londres', '12e', 'Brésilienne', 'Espagne',
       'Britannique', 'États-Unis', '', 'Cubaine  Américaine', 'Lange',
       'britannique', 'Bouches-du-Rhône', 'thaïlandaise', 'Toronto',
       'CongolaiseFrançaise', 'Italie', 'français', 'Canada', 'York',
       'Texas', 'Britannique  Norvègien', 'Tetney', 'Belge',
       'Lancashire,Angleterre', 'Américaine', 'Pays-Bas'], dtype=object)

In [ ]:
sql = "INSERT INTO artist(nameArtist, birth, city, country) VALUES(%s)"

cur.execute(sql, ("",))

In [137]:
pip install lyricsgenius --user


  Using cached https://files.pythonhosted.org/packages/0d/32/be32f6922f70fd1b9900b50b228f6585cd60a96bdf03589df738f627d388/lyricsgenius-3.0.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [138]:
from lyricsgenius import Genius

genius = Genius(token)

NameError: name 'token' is not defined